In [1]:
from pymongo import MongoClient
from bson.objectid import ObjectId

In [2]:
client = MongoClient('act4dgem.cse.iitd.ac.in', serverSelectionTimeoutMS=600000)
db = client['eventwise_media-db']

In [3]:
resolved_entities_1 = db.get_collection('all_media_entities_resolved_final')

In [4]:
resolved_entities_2 = db.get_collection('all_media_entities_resolved_final3')

In [5]:
with open('./relationships_works_in.txt') as fil:
    lines = fil.readlines()
instances = [l[10:].strip().split('\t')[:2] for l in lines if l.startswith('instance')]
sentences = [l[10:].strip() for l in lines if l.startswith('sentence')]

In [6]:
lines[1]

'sentence: Hong Kong: Asian insurer <ORG>5982d07f9855b7221e184316</ORG> , the world\'s second largest by m-cap, will scale up its business in India undeterred by either the boardroom battle at its joint venture partner <ORG>5982d07f9855b7221e184314</ORG>>5982d07f9855b7221e184315</PER> or the heat and dust of demonetisation . AIA, which was spun off from the <ORG>5982d07f9855b7221e18431b</ORG> through a nearly $18-billion IPO in 2010, has emerged as the largest pan-Asian insurer.In an interview to TOI, <PER>5982d07f9855b7221e184311</PER> , regional chief executive, AIA, said that the Indian joint venture Tata AIA Life Insurance has moved up ten positions in rankings to become the sixth largest life insurance company. "Looking at the Tata brand, we should be among the top three players. In the life business, we had some stop-starts. A key part of that was not having a bancassurance partner. But we turned that corner by partnering <ORG>5982d07f9855b7221e184319</ORG> and <ORG>5982d0959855b

In [8]:
name = []
name_uuid = []
org = []
org_uuid = []
sen = []
for i,inst in enumerate(instances):
    res = resolved_entities_1.find_one({'_id':ObjectId(inst[0]), 'graphid':{'$exists':True}, 'type':'Person'})
    s = sentences[i]
    if res:
        name.append(res['stdName'])
        name_uuid.append(res['graphid'])
        s = s.replace(inst[0], res['stdName'])
    else:
        continue
    res = resolved_entities_2.find_one({'_id':ObjectId(inst[1]), 'graphid':{'$exists':True}})
    if res:
        org.append(res['stdName'])
        org_uuid.append(res['graphid'])
        s = s.replace(inst[1], res['stdName'])
    else:
        del name[-1]
        continue
    sen.append(s)

In [9]:
len(name) == len(org)

True

In [10]:
list(zip(name, org))

[('ASHOK BHANUSHALI TRIKAMJI', 'GUJARAT POLY AVX ELECTRONICS LTD.'),
 ('ANAND SINGH RATTANDEEP',
  'HOTEL AND RESTAURANT ASSOCIATION OF NORTHERN INDIA ..'),
 ('GADICHERLA SUBBA RAO', 'SVS PRODUCTS PRIVATE LIMITED'),
 ('SIDHI PARKASH JAIN', 'RELIANCE INFRATEL LIMITED'),
 ('MOHAN KAMESHWARAN', 'MADRAS ENT RESEARCH FOUNDATION PRIVATE LIMITED'),
 ('MOHAN KAMESHWARAN', 'MADRAS ENT RESEARCH FOUNDATION PRIVATE LIMITED'),
 ('NUGGEHALLI GEETHAPRABHU JAYASIMHA', 'HUMANE SOCIETY. INTERNATIONAL: INDIA'),
 ('KAILASH KUMAR GOYAL', 'EXPORT PROMOTION COUNCIL FOR HANDICRAFTS'),
 ('JWALANT SURESHBHAI SHAH', 'ENVIRONMENTAL RESOURCES FOUNDATION'),
 ('JAYASHREE VENKATRAMAN', 'TRACTORS AND FARM EQUIPMENT LIMITED.'),
 ('DEEPAK MOOLCHAND LAHOTI', 'BAJAJ FINSERV LIMITED'),
 ('chandrasekaran.m', 'CONFEDERATION OF INDIAN INTERNET COMMERCE'),
 ('Lochan Singh', 'GLOBAL AUTOTECH LIMITED'),
 ('LAKSHMIKANTHAN KRISHNAN',
  'RENAULT NISSAN TECHNOLOGY & BUSINESS CENTRE INDIA PRIVATE LIMITED'),
 ('NILADRI SEKHAR ROY', 'M

In [11]:
import csv

In [16]:
with open('relatedInstancesWorksIn.csv', 'w') as fil:
    writer = csv.writer(fil)
    for i in range(len(name)):
        writer.writerow([name[i], name_uuid[i], org[i], org_uuid[i], sen[i]])

In [17]:
len(name)

294

In [18]:
len(sen)

294

In [19]:
len(org)

294

In [20]:
with open('relatedInstancesInGraphDb_WorksIn.txt', 'w') as fil:
    for i in range(len(sen)):
        print(name[i], '\t' , org[i], '\n', sen[i], '\n', '-'*50, file=fil)